# Notebook summary
## Goal
* This notebook is a reproduction of the preprocessing steps involved in the creation of the dataset. It follows what was done in the following notebook: https://github.com/KIZI/evaluation-of-comprehensibility/blob/master/clustering_analysis_v2/LINVILLWARREN-AlternativeB-full-100features.ipynb
* If you want mroe information on this preprocessing please follow the link above.

# Imports

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!git clone https://github.com/fivethirtyeight/russian-troll-tweets.git

import pandas as pd
import re
from collections import OrderedDict
from nltk import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
import pandas as pd
from sqlalchemy import create_engine, inspect

# Create the dataset

In [ ]:
dataset_to_process = "LINVILL" 

engine = create_engine(f'sqlite:///{dataset_to_process}2full.sqlite')
connection = engine.connect()
print(engine.table_names())
inspector = inspect(connection)

[]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  after removing the cwd from sys.path.


In [ ]:
if dataset_to_process == "LINVILL":
    CLUSTERS_TO_TRY=[5,6]
    MINIMUM_TWEETS_PER_ACCOUNT=100 # use only accounts with greater than this tweets
    PROCESS_MAX_TWEETS_PER_ACCOUNT = 100 #100
    RANDOMIZE_TWEET_ORDER=True
    MIN_DF = 0.4 #as in the original experiment for user study
    MAX_FEATURES= None #as in the original experiment for user study
    overrideClusterSelection = "random6cosine"
    tablename= "tweets"
    if not tablename in engine.table_names():
        for i in range(1,14):
            inputFile = f"/content/russian-troll-tweets/IRAhandle_tweets_{i}.csv"
            print(inputFile)
            df = pd.read_csv(inputFile)
            #print("randomly selecting subset of data")
            #df=df.sample(random_state=42,frac=0.01)
            df.to_sql(name=tablename, con=connection, if_exists='append', index=True)
    else:
        print("already present in database")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  # Remove the CWD from sys.path while we load stuff.


/content/russian-troll-tweets/IRAhandle_tweets_1.csv
/content/russian-troll-tweets/IRAhandle_tweets_2.csv
/content/russian-troll-tweets/IRAhandle_tweets_3.csv
/content/russian-troll-tweets/IRAhandle_tweets_4.csv
/content/russian-troll-tweets/IRAhandle_tweets_5.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,15,20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


/content/russian-troll-tweets/IRAhandle_tweets_6.csv
/content/russian-troll-tweets/IRAhandle_tweets_7.csv
/content/russian-troll-tweets/IRAhandle_tweets_8.csv
/content/russian-troll-tweets/IRAhandle_tweets_9.csv
/content/russian-troll-tweets/IRAhandle_tweets_10.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


/content/russian-troll-tweets/IRAhandle_tweets_11.csv
/content/russian-troll-tweets/IRAhandle_tweets_12.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10,20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


/content/russian-troll-tweets/IRAhandle_tweets_13.csv


In [ ]:
# Counts the number of tweets per author (only counts tweets in English)
tablename= "tweets_authors"
troll_sql = """drop table IF EXISTS tweets_authors;"""
cursor = connection.execute(troll_sql)
troll_sql = """
create table tweets_authors as
select
author, account_category, count(*) as tweet_cnt
from tweets
where retweet = 0 and (language = 'English' or language = 'en')
group by author;"""
cursor = connection.execute(troll_sql)

In [ ]:
# Only select the authors that have more than MINIMUM_TWEETS_PER_ACCOUNT
cursor = connection.execute(f"select author from tweets_authors where tweet_cnt >= {MINIMUM_TWEETS_PER_ACCOUNT}")
authors = [author for (author, ) in cursor]
len(authors)

578

In [ ]:
tablename= "tweets_sample"
troll_sql = """drop table IF EXISTS tweets_sample;"""
cursor = connection.execute(troll_sql)

if RANDOMIZE_TWEET_ORDER:
    randSQL="order by random()"
else:
    randSQL = ""
if not tablename in engine.table_names(): # this line is not needed if the drop is present
    connection.execute(f"CREATE TABLE tweets_sample (content varchar({PROCESS_MAX_TWEETS_PER_ACCOUNT * 150}), author varchar(255), account_category varchar(255));")
    for author in authors:
        print(author)
        connection.execute(f"""
        insert into tweets_sample (content, author, account_category)
        select content, author, account_category from tweets
        where author = '{author}' 
        {randSQL} limit {PROCESS_MAX_TWEETS_PER_ACCOUNT};
        """)
        r= connection.execute(f"""
        select count(*) from tweets_sample
        """)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  if __name__ == '__main__':


10_GOP
4MYSQUAD
AANTIRACIST
ABIGAILSSILK
ABIISSROSB
ABMERRLINS
ACAB_ZONE
ACAPARELLA
ACEJINEV
ADAMCHAPMANJR
ADDIE_HOL
ADELE_BROCK
ADNNELSTR
ADRGREERR
ADRIASNSS
ADRIENNE_GG
AFEELIUTR
AIDEN7757
ALANISSTRS
ALAXXATR
ALBERTMORENMORE
ALBUQUERQUEON
ALDRICH420
ALECMOOOOODY
ALEXXDRTRR
ALIISTRR
ALLBERRTOSS
AMANDAVGREEN
AMBBERTHTT
AMBERLINETR
AMBERRPAR
AMCONVOICE
AMELIEBALDWIN
AMELINAASTR
AMERICANALBERT
AMIISTRS
AMYYSLEGENTR
ANAAISLEC
ANAALESSIS
ANASTASSITR
ANCARICTRS
ANDEERLWR
ANDIOSMARRTRUMP
ANDRISNABRT
ANDRREETRT
ANDYDREWDANIELS
ANDYHASHTAGGER
ANGEELISTR
ANGELISTRR
ANGISDOOYL
ANGTHEESTR
ANIIANTRS
ANISACRRUTR
ANISILOPE
ANISSNOVOTR
ANNAMINGT
ANNIEPOSHES
ANNIJONSTR
ANTONHAYHAY
ANTONRUMSS
ANZGRI
APRRILSBRUSS
ARABMTR
ARAGARTRR
ARCHIEOLIVERS
ARIAANST
ARIEENSLPE
ARISGRAYS
ARM_2_ALAN
ARTEMSSFE
ASHLEEYWES
ASWWIMMORRIS
ATLANTA_ONLINE
AURRLISTR
AUSTINLOVESBEER
AYLEENSCLAR
AZUUMARSS
BABYJANNYMEOW
BALTIMORE0NLINE
BAMMBJOLTR
BAOBAEHAM
BARBARAFORTRUMP
BARELYBARBOZA
BARRLAUTRS
BATONROUGEVOICE
BEAUTYMERCELA
BEC

In [ ]:
sql = 'select author, content, account_category from tweets_sample order by author'
cursor= connection.execute(sql)
#result = cursor.fetchall()
documents_dict = OrderedDict()
true_labels = []
# merge all tweets by one author into one entry 
authors=0
tweets=0
for (author, content, account_category) in cursor:
    if dataset_to_process == "MANNHEIM":
        if account_category =="0":
            account_category="real news"
        else:
            account_category="fake news"
    if author in documents_dict:
        if content != None:
            tweets=tweets+1
            documents_dict[author] += '\n' + content
            #if the tweet is in a different category than previously seen for this author
            #then reassign the author to the "mixed" category
            #this is applicable only for the Mannheim dataset, where
            #for several accounts some tweets are marked as 0 and some as 1
            if true_labels[len(true_labels)-1]!="mixed" and account_category != true_labels[len(true_labels)-1]:
                true_labels[len(true_labels)-1] = "mixed"
        else:
            print("Skipped empty content")
    else:
        authors=authors+1
        tweets= tweets +1
        true_labels.append(account_category)
        documents_dict[author] = content
print("authors " + str(authors))
print("tweets " + str(tweets))

authors 578
tweets 57800


In [ ]:
# create a pandas dataframe
data = {'authors': list(documents_dict.keys()), 'Tweets': list(documents_dict.values()), 'Labels': true_labels}
df = pd.DataFrame(data)
print(df.shape)
df.head() 

(578, 3)


,authors,Tweets,Labels
0,10_GOP,'@realDonaldTrump We have your back Mr. Presid...,RightTroll
1,4MYSQUAD,fuck that older brother is in trooooooouble ht...,LeftTroll
2,AANTIRACIST,Kneel down and pray & know #God is has already...,LeftTroll
3,ABIGAILSSILK,#RetireASongOrBand Silver Surfin' USA\n#Vuln...,HashtagGamer
4,ABIISSROSB,#abi Trump’s Approval Rating LEAPS! https://t....,RightTroll


In [ ]:
# save the dataset
df.to_csv("100_tweets.csv")

/content/drive/My Drive/Research Assistant/Paper
